# Titanic Challenge Kaggle.com

### Aqui faço um pequeno código para o desafio do titanic(https://www.kaggle.com/c/titanic).
### O desafio consiste em predizer os sobreviventes e não-sobreviventes do acidente levando em consideração algumas informações dos passageiros.


### Importando as bibliotecas necessárias

In [16]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

### Carregando os dados de treinamento e separando as variaveis de treinamento: X_train e y_train

In [17]:
dataset_train = pd.read_csv("train.csv")
dataset_train = dataset_train.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis = 1)
X_train = dataset_train.drop(['Survived'], axis = 1)
y_train = dataset_train['Survived'].values

### Carregando os dados de Teste e separando as variaveis de teste: X_test e y_test

In [18]:
dataset_test = pd.read_csv("test.csv")
dataset_test = dataset_test.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis = 1)
X_test = dataset_test

### Em seguida, criamos 2 novas variaveis a partir das existentes: Tamanho da Família(FamilySize) e uma variavel binária para indicar se o passageiro estava ou não sem familiares na viagem(IsAlone)

In [19]:
X_train['FamilySize'] = X_train['SibSp'] + X_train['Parch'] + 1
X_train['IsAlone'] = 1
X_train['IsAlone'].loc[X_train['FamilySize'] > 1] = 0

X_test['FamilySize'] = X_test['SibSp'] + X_test['Parch'] + 1
X_test['IsAlone'] = 1
X_test['IsAlone'].loc[X_test['FamilySize'] > 1] = 0       

### Agora temos que lidar com os dados faltantes no dataset, depois de alguns testes escolhi preenche-los com mediana e moda da coluna correspondente

In [20]:
X_train['Age'].fillna(X_train['Age'].median(), inplace = True)
X_train['Embarked'].fillna(X_train['Embarked'].mode()[0], inplace = True)
X_train['Fare'].fillna(X_train['Fare'].median(), inplace = True)

X_test['Age'].fillna(X_test['Age'].median(), inplace = True)
X_test['Embarked'].fillna(X_test['Embarked'].mode()[0], inplace = True)
X_test['Fare'].fillna(X_test['Fare'].median(), inplace = True)

### Agora precisamos lidar com as variáveis categoricas do dataset, já que a maioria dos algoritmos só lidam com números

In [21]:
label = LabelEncoder()
X_train['Sex'] = label.fit_transform(X_train['Sex'])
X_train['Embarked'] = label.fit_transform(X_train['Embarked'])
onehotencoder = OneHotEncoder(categorical_features=[6])
X_train = onehotencoder.fit_transform(X_train).toarray()

In [22]:
label = LabelEncoder()
X_test['Sex'] = label.fit_transform(X_test['Sex'])
X_test['Embarked'] = label.fit_transform(X_test['Embarked'])
onehotencoder = OneHotEncoder(categorical_features=[6])
X_test = onehotencoder.fit_transform(X_test).toarray()

### Precisamos agora normalizar os dados para acelerar o processo de treinamento e entre outras vantagens

In [23]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Vamos agora treinar o classificador, o melhor classificador testado anteriormente foi o XGBOOST algoritmo baseado na técnica de Boosting como o próprio nome já indica

In [24]:
classifier = XGBClassifier(n_estimators=120)
classifier.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=120,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

### Após o treinamento, guardamos o resultado de saída do modelo para o nosso conjunto de teste na variável y_pred

In [25]:
y_pred = classifier.predict(X_test)

### Para avaliar o modelo, utilizo k-fold cross validation escolhendo k=10.Essa técnica é uma das técnicas mais utilizadas por cientistas de dados

In [27]:
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
mean = accuracies.mean()
std = accuracies.std()
print("Média de acurácia do algoritmo: %f Desvio Padrão: %f" % (mean, std))

Média de acurácia do algoritmo: 0.827237 Desvio Padrão: 0.034067


### Essa taxa de acerto é satisfatória, portanto, enviaremos o csv das predições para a submissão da competição

In [28]:
temp = pd.read_csv('test.csv')
PassengerId = temp['PassengerId']
Submission = pd.DataFrame({ 'PassengerId': PassengerId,
                            'Survived': y_pred })
Submission.to_csv("Submission.csv", index=False)